Part A asks to create an artificial 3D vector field with 2D horizontal vectors.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
print('Hello Git')

Hello Git


Part B asks to create an artificial vector field. This will be a 3 dimensional field with 2d horizontal vecotors. The artificial vector field will be composed by creating a column vector that is copied to the other places. There will be two profiles selected, the log law and power law profiles. An Ekman spiral will also be added to the vector. Variation will be added with up to a random 10% "kick".

In [8]:
''' A 1 km cube is going to be used with a 1 m resolution.
The column vector will be a 1000 by 2 matrix. This is 1000 rows with 2 columns. Each row will the vector at a discrete height. The two columns will correspond to a u and v component of the vector.'''
height = 1000 # number of meters in height
resolution = 1 # resolution in meters
ColumnVecSize = (height//resolution,2) # size of column vector using the height and resolution
# Allocation of memory for column vector
ColumnVector = np.zeros(ColumnVecSize, dtype=float) # float data type designation

y_location = np.arange(0,height,resolution,dtype=float) # define the heights of each vector

U_inf = 12 # speed of wind outside of boundary incfluence

rho = 1.293 # density of dry air at STP kg/m^3

u_friction_velocity = np.sqrt(wall_shear / rho)
viscous_length_scale = nu / u_friction_velocity
y_plus = y_location / viscous_length_scale
Karman_const = 0.41 # Karman constant for log law
E_const = 7.76 # offset constant
U_plus = 1/Karman_const * np.log(y_plus*E_const)

print('Cool this works')